# Building cnn from scratch without using transformed data

In [1]:
from mlxtend.data import loadlocal_mnist
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
X_train,Y_train=loadlocal_mnist(images_path='train-images-idx3-ubyte',labels_path='train-labels-idx1-ubyte')
X_test,Y_test=loadlocal_mnist(images_path='t10k-images-idx3-ubyte',labels_path='t10k-labels-idx1-ubyte')

In [7]:
#plt.imshow(X_train[img_index])

In [3]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [4]:
X_train.shape

(60000, 784)

In [5]:
Y_train = np.eye(10)[Y_train.astype('int32')]

In [6]:
Y_test = np.eye(10)[Y_test.astype('int32')]

In [8]:
Y_train.shape

(60000, 10)

In [9]:
# Placeholder variable for the input images
x = tf.placeholder(tf.float32, shape=[None, 28*28], name='X')
# Reshape it into [num_images, img_height, img_width, num_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Placeholder variable for the true labels associated with the images
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

Instructions for updating:
Use the `axis` argument instead


In [10]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, name):
    
    with tf.variable_scope(name) as scope:
        # Shape of the filter-weights for the convolution
        shape = [filter_size, filter_size, num_input_channels, num_filters]

        # Create new weights (filters) with the given shape
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))

        # Create new biases, one for each filter
        biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))

        # TensorFlow operation for convolution
        layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')

        # Add the biases to the results of the convolution.
        layer += biases
        
        return layer, weights

In [11]:
def new_pool_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.max_pool(value=input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        return layer

In [12]:
def new_relu_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.relu(input)
        
        return layer

In [13]:
def new_fc_layer(input, num_inputs, num_outputs, name):
    
    with tf.variable_scope(name) as scope:

        # Create new weights and biases.
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev=0.05))
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
        
        # Multiply the input and weights, and then add the bias-values.
        layer = tf.matmul(input, weights) + biases
        
        return layer

In [14]:
# Convolutional Layer 1
layer_conv1, weights_conv1 = new_conv_layer(input=x_image, num_input_channels=1, filter_size=5, num_filters=6, name ="conv1")

# Pooling Layer 1
layer_pool1 = new_pool_layer(layer_conv1, name="pool1")

# RelU layer 1
layer_relu1 = new_relu_layer(layer_pool1, name="relu1")

# Convolutional Layer 2
layer_conv2, weights_conv2 = new_conv_layer(input=layer_relu1, num_input_channels=6, filter_size=5, num_filters=16, name= "conv2")

# Pooling Layer 2
layer_pool2 = new_pool_layer(layer_conv2, name="pool2")

# RelU layer 2
layer_relu2 = new_relu_layer(layer_pool2, name="relu2")

# Flatten Layer
num_features = layer_relu2.get_shape()[1:4].num_elements()
layer_flat = tf.reshape(layer_relu2, [-1, num_features])

# Fully-Connected Layer 1
layer_fc1 = new_fc_layer(layer_flat, num_inputs=num_features, num_outputs=128, name="fc1")

# RelU layer 3
layer_relu3 = new_relu_layer(layer_fc1, name="relu3")

# Fully-Connected Layer 2
layer_fc2 = new_fc_layer(input=layer_relu3, num_inputs=128, num_outputs=10, name="fc2")

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
 #Use Softmax function to normalize the output
with tf.variable_scope("Softmax"):
    y_pred = tf.nn.softmax(layer_fc2)
    y_pred_cls = tf.argmax(y_pred, dimension=1)

In [16]:
# Use Cross entropy cost function
with tf.name_scope("cross_ent"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)
    cost = tf.reduce_mean(cross_entropy)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [17]:
# Use Adam Optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [18]:
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
# Initializing the variables
init = tf.global_variables_initializer()

In [20]:
training_iters = 200 
learning_rate = 0.001 
batch_size = 128

In [21]:
with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    for i in range(training_iters):
        for batch in range(len(X_train)//batch_size):
            batch_x = X_train[batch*batch_size:min((batch+1)*batch_size,len(X_train))]
            batch_y = Y_train[batch*batch_size:min((batch+1)*batch_size,len(Y_train))]
            opt = sess.run(optimizer, feed_dict={x: batch_x,y_true: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y_true: batch_y})
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
        print("Optimization Finished!")

        # Calculate accuracy for all 10000 mnist test images
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: X_test,y_true : Y_test})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()

Iter 0, Loss= 0.716035, Training Accuracy= 0.72656
Optimization Finished!
Testing Accuracy: 0.73210
Iter 1, Loss= 0.573506, Training Accuracy= 0.76562
Optimization Finished!
Testing Accuracy: 0.77230
Iter 2, Loss= 0.475227, Training Accuracy= 0.77344
Optimization Finished!
Testing Accuracy: 0.79790
Iter 3, Loss= 0.420315, Training Accuracy= 0.79688
Optimization Finished!
Testing Accuracy: 0.81370
Iter 4, Loss= 0.383841, Training Accuracy= 0.81250
Optimization Finished!
Testing Accuracy: 0.82600
Iter 5, Loss= 0.357911, Training Accuracy= 0.82812
Optimization Finished!
Testing Accuracy: 0.83330
Iter 6, Loss= 0.340631, Training Accuracy= 0.83594
Optimization Finished!
Testing Accuracy: 0.83920
Iter 7, Loss= 0.326186, Training Accuracy= 0.85938
Optimization Finished!
Testing Accuracy: 0.84270
Iter 8, Loss= 0.314003, Training Accuracy= 0.86719
Optimization Finished!
Testing Accuracy: 0.84690
Iter 9, Loss= 0.304556, Training Accuracy= 0.87500
Optimization Finished!
Testing Accuracy: 0.85040


Testing Accuracy: 0.89910
Iter 82, Loss= 0.095653, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy: 0.89940
Iter 83, Loss= 0.095186, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy: 0.89980
Iter 84, Loss= 0.094342, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy: 0.90100
Iter 85, Loss= 0.093214, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy: 0.90150
Iter 86, Loss= 0.092575, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy: 0.90140
Iter 87, Loss= 0.092086, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy: 0.90200
Iter 88, Loss= 0.091259, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy: 0.90210
Iter 89, Loss= 0.091012, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy: 0.90190
Iter 90, Loss= 0.089561, Training Accuracy= 0.97656
Optimization Finished!
Testing Accuracy: 0.90200
Iter 91, Loss= 0.088737, Training Accuracy= 0.97656
Optimization 

Testing Accuracy: 0.90720
Iter 163, Loss= 0.060911, Training Accuracy= 0.98438
Optimization Finished!
Testing Accuracy: 0.90760
Iter 164, Loss= 0.060556, Training Accuracy= 0.98438
Optimization Finished!
Testing Accuracy: 0.90760
Iter 165, Loss= 0.059722, Training Accuracy= 0.98438
Optimization Finished!
Testing Accuracy: 0.90770
Iter 166, Loss= 0.060198, Training Accuracy= 0.98438
Optimization Finished!
Testing Accuracy: 0.90790
Iter 167, Loss= 0.059475, Training Accuracy= 0.98438
Optimization Finished!
Testing Accuracy: 0.90760
Iter 168, Loss= 0.059544, Training Accuracy= 0.98438
Optimization Finished!
Testing Accuracy: 0.90790
Iter 169, Loss= 0.059002, Training Accuracy= 0.98438
Optimization Finished!
Testing Accuracy: 0.90770
Iter 170, Loss= 0.058931, Training Accuracy= 0.98438
Optimization Finished!
Testing Accuracy: 0.90780
Iter 171, Loss= 0.058267, Training Accuracy= 0.98438
Optimization Finished!
Testing Accuracy: 0.90720
Iter 172, Loss= 0.058295, Training Accuracy= 0.98438
Opt